<a href="https://colab.research.google.com/github/fairuznawar122/Network-Intrusion-Detection-System/blob/main/NIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd

# Specify the path to your file
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/LDAP.csv'
file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/MSSQL.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/NetBIOS.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/Portmap.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/Syn.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/UDP.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/UDPLag.csv'

# Load the dataset into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/MSSQL.csv'

In [19]:
df.columns = df.columns.str.strip()
print(df.columns)

Index(['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Le

In [20]:
# Now you can access the 'Label' column
label_counts = df['Label'].value_counts()

# Display the counts
print(label_counts)


Label
LDAP       1905191
NetBIOS     202919
BENIGN        5124
Name: count, dtype: int64


# Feature Names Used

In [21]:
feature_names = ['Fwd Packet Length Max',
                 'Fwd Packet Length Min',
                 'Min Packet Length',
                 'Max Packet Length',
                 'Average Packet Size',
                 'Fwd Packets/s',
                 'Fwd Header Length',
                 'Fwd Header Length.1',
                 'min_seg_size_forward',
                 'Total Length of Fwd Packets',
                 'Fwd Packet Length Std',
                 'Flow IAT Min',
                 'Subflow Fwd Bytes',
                 'Destination Port',
                 'Protocol',
                 'Packet Length Std',
                 'Flow Duration',
                 'Fwd IAT Total',
                 'ACK Flag Count',
                 'Init_Win_bytes_forward',
                 'Flow IAT Mean',
                 'Flow IAT Max',
                 'Fwd IAT Mean',
                 'Fwd IAT Max',
                 'Label']
len(feature_names)

25

In [27]:
df= df[feature_names]


# Resampling Data to Balance it

In [28]:
import pandas as pd
from sklearn.utils import resample

# Creating the initial dataset


# Determine the target number of samples (the number of 'BENIGN' samples)
target_size = df[df['Label'] == 'BENIGN'].shape[0]

# Resample the dataset
ldap_resampled = resample(df[df['Label'] == 'LDAP'],
                          replace=True,     # Sample with replacement
                          n_samples=target_size, # Match the 'BENIGN' count
                          random_state=42)    # For reproducibility

netbios_resampled = resample(df[df['Label'] == 'NetBIOS'],
                             replace=True,
                             n_samples=target_size,
                             random_state=42)

benign_resampled = df[df['Label'] == 'BENIGN']  # BENIGN already matches the target size

# Combine all resampled categories back into one DataFrame
resampled_df = pd.concat([ldap_resampled, netbios_resampled, benign_resampled])

# Shuffle the DataFrame to mix the samples
resampled_df = resampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new distribution
print(resampled_df['Label'].value_counts())


Label
NetBIOS    5124
BENIGN     5124
LDAP       5124
Name: count, dtype: int64


In [30]:
resampled_df = resampled_df.reset_index(drop=True)
resampled_df

,Fwd Packet Length Max,Fwd Packet Length Min,Min Packet Length,Max Packet Length,Average Packet Size,Fwd Packets/s,Fwd Header Length,Fwd Header Length.1,min_seg_size_forward,Total Length of Fwd Packets,...,Packet Length Std,Flow Duration,Fwd IAT Total,ACK Flag Count,Init_Win_bytes_forward,Flow IAT Mean,Flow IAT Max,Fwd IAT Mean,Fwd IAT Max,Label
0,229.0,229.0,229.0,229.0,343.5,2.000000e+06,16,16,8,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,NetBIOS
1,229.0,229.0,229.0,229.0,343.5,2.000000e+06,16,16,8,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,NetBIOS
2,0.0,0.0,0.0,0.0,0.0,3.772105e+00,128,128,32,0.0,...,0.000000,1060416,1023970.0,0,296,212083.2,1023962.0,341323.333333,1023962.0,BENIGN
3,229.0,229.0,229.0,229.0,343.5,2.000000e+06,16,16,8,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,NetBIOS
4,1472.0,1472.0,1472.0,1472.0,2208.0,2.000000e+06,28,28,14,2944.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,LDAP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15367,229.0,229.0,229.0,229.0,343.5,2.000000e+06,64,64,32,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,NetBIOS
15368,53.0,6.0,6.0,53.0,56.0,2.000000e+06,40,40,20,59.0,...,27.135463,1,1.0,0,37800,1.0,1.0,1.000000,1.0,BENIGN
15369,229.0,229.0,229.0,229.0,343.5,2.000000e+06,40,40,20,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,NetBIOS
15370,1472.0,1472.0,1472.0,1472.0,2208.0,2.000000e+06,40,40,20,2944.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,LDAP


In [32]:
# Replace 'NetBIOS' with 1 and 'BENIGN' with 0 in the 'Category' column
resampled_df['Label'] = resampled_df['Label'].replace({'NetBIOS': 1, 'LDAP':1, 'BENIGN': 0})

# Display the updated DataFrame
print(resampled_df['Label'].value_counts())


Label
1    10248
0     5124
Name: count, dtype: int64


<ipython-input-32-b2ba4ff1a8c9>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  resampled_df['Label'] = resampled_df['Label'].replace({'NetBIOS': 1, 'LDAP':1, 'BENIGN': 0})


# Creating the Main Dataset

In [33]:
resampled_df

,Fwd Packet Length Max,Fwd Packet Length Min,Min Packet Length,Max Packet Length,Average Packet Size,Fwd Packets/s,Fwd Header Length,Fwd Header Length.1,min_seg_size_forward,Total Length of Fwd Packets,...,Packet Length Std,Flow Duration,Fwd IAT Total,ACK Flag Count,Init_Win_bytes_forward,Flow IAT Mean,Flow IAT Max,Fwd IAT Mean,Fwd IAT Max,Label
0,229.0,229.0,229.0,229.0,343.5,2.000000e+06,16,16,8,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,1
1,229.0,229.0,229.0,229.0,343.5,2.000000e+06,16,16,8,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,1
2,0.0,0.0,0.0,0.0,0.0,3.772105e+00,128,128,32,0.0,...,0.000000,1060416,1023970.0,0,296,212083.2,1023962.0,341323.333333,1023962.0,0
3,229.0,229.0,229.0,229.0,343.5,2.000000e+06,16,16,8,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,1
4,1472.0,1472.0,1472.0,1472.0,2208.0,2.000000e+06,28,28,14,2944.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15367,229.0,229.0,229.0,229.0,343.5,2.000000e+06,64,64,32,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,1
15368,53.0,6.0,6.0,53.0,56.0,2.000000e+06,40,40,20,59.0,...,27.135463,1,1.0,0,37800,1.0,1.0,1.000000,1.0,0
15369,229.0,229.0,229.0,229.0,343.5,2.000000e+06,40,40,20,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,1
15370,1472.0,1472.0,1472.0,1472.0,2208.0,2.000000e+06,40,40,20,2944.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,1
